In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
## library import
%matplotlib inline

# for image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

In [0]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
ls

gdrive/  sample_data/


In [0]:
cd gdrive

/content/gdrive


In [0]:
cd ..

/content


### Data Load

In [0]:
train_dir = 'gdrive/My Drive/Colab Notebooks/data/train/train_dir'
test_dir = 'gdrive/My Drive/Colab Notebooks/data/test'
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

In [0]:
# subset of the dataset to make faster to train
from torch.utils.data.sampler import SubsetRandomSampler
training_samples = 300
test_samples = 300

train_sampler = SubsetRandomSampler(np.arange(training_samples, dtype=np.int64))
test_sampler = SubsetRandomSampler(np.arange(test_samples, dtype=np.int64))

In [0]:
# For traing data we used Data Augmentation of the form of transforms that
# randomly rotate, scale and crop, then flip images
trans = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# For test data we don't use any form of Data Augmentation and just used
# resize and center crop to adjust our images for input


test_trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


# Define train and test folders located on google drive
train_data = torchvision.datasets.ImageFolder(root=train_dir, transform=trans)

test_data = torchvision.datasets.ImageFolder(root=test_dir, transform=test_trans)

#Define dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64,
                                          sampler=train_sampler, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64,
                                          sampler=test_sampler, num_workers=2)

In [0]:
#폴더명을 label로 알려줌
print(train_data.classes)

['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


### Building Model

In [0]:
from torchvision import models
net = models.vgg19().to(device)
#net = models.resnet50().to(device)
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)

In [0]:
import torch.nn as nn
# Transfer learning 사용으로 classifier만 training
for param in net.parameters():
    param.requires_grad = True
num_ftrs = net.classifier[0].in_features


# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
net.fc = nn.Linear(num_ftrs, 2)

In [0]:
criterion = nn.CrossEntropyLoss().to(device)

# Observe that only parameters of final layer are being optimized
#optimizer_conv = optim.SGD(net.fc.parameters(), lr=0.001, momentum=0.9)
# 모든 매개변수들이 최적화되었는지 관찰
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

# 5 에폭마다 0.1씩 학습율 감소
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [0]:
def train_model(epoch):
    # print(len(trainloader))
    # loop over the dataset multiple times
      net.train()
      running_loss = 0.0
      running_corrects = 0
      lr_sche.step()
      for i, data in enumerate(trainloader, 0):
          # get the inputs
          inputs, labels = data
          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)
          
          # 역전파 + 최적화
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)
          
      epoch_loss = running_loss / len(trainloader)
      epoch_acc = running_corrects.double() / len(trainloader)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'train', epoch_loss, epoch_acc))
 

      print('Finished Training')

In [0]:
def test_model(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    running_corrects = 0
    with torch.no_grad():
        for i,data in enumerate(testloader):
            inputs, labels = data
            inputs, targets = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(testloader)
        epoch_acc = running_corrects.double() / len(testloader)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'test', epoch_loss, epoch_acc))

    # Save checkpoint.
    acc = 100.*epoch_acc
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


In [0]:
import copy
start_epoch = 0
net = net.to(device)
for epoch in range(start_epoch, start_epoch+25):
  best_model_wts = copy.deepcopy(net.state_dict())
  best_acc = 0.0
  train_model(epoch)
  test_model(epoch)

train Loss: 299.3606 Acc: 37.0000
Finished Training


IndexError: ignored